# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

In [1]:
config_list = [
  {
    "model": "llama3.1:8b",
    "base_url": "http://192.168.9.122:11434/v1",
    "api_key": "ollama",
  }
]

config_list = [
  {
    "model": "Qwen2.5-7B-Instruct",
    "base_url": "http://192.168.9.122:8000/v1",
    "api_key": "token-abc123",
  }
]

llm_config = {"config_list": config_list}

## Define an AutoGen agent

In [2]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

/home/ncs/miniconda3/envs/autogen_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-08 09:34:44,451	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/ncs/miniconda3/envs/autogen_test/lib/python3.11/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [7]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

{'content': "Sure, here's a light joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': []}


In [8]:
# generate_reply这个接口没有记忆功能
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

{'content': "Sure, I'd be happy to repeat a joke, but you haven't provided a specific joke yet. Could you please share the joke you'd like me to repeat?", 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': []}


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [4]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [5]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Glad to keep the laughs coming. Why did the tomato turn red? Because it saw the salad dressing! 😂 Now, are you a leftie or a rightie? Because I heard lefties have an advantage in stand-up comedy because they're used to dealing with all the awkward left-handed equipment! 😜

--------------------------------------------------------------------------------
joe (to cathy):

Well, I'm right-handed, but I guess I must be pretty flexible because I can make both left-handed and right-handed jokes! 🤚✍️ 👉✍️

--------------------------------------------------------------------------------
cathy (to joe):

You go, Joe! Flexibility is key, whether you're spinning a joke or a plate. Here’s one for you: Why did the scarecrow win an award? Because he was outstanding in his field! 😂 And speaking of fields, have you ever tried standing 

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [9]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': 'Hey Joe! Glad to keep the laughs coming. Why did the tomato turn '
             'red? Because it saw the salad dressing! 😂 Now, are you a leftie '
             'or a rightie? Because I heard lefties have an advantage in '
             "stand-up comedy because they're used to dealing with all the "
             'awkward left-handed equipment! 😜',
  'role': 'user',
  'tool_calls': []},
 {'content': "Well, I'm right-handed, but I guess I must be pretty flexible "
             'because I can make both left-handed and right-handed jokes! 🤚✍️ '
             '👉✍️',
  'role': 'assistant',
  'tool_calls': []},
 {'content': "You go, Joe! Flexibility is key, whether you're spinning a joke "
             'or a plate. Here’s one for you: Why did the scarecrow win an '
             'award? Because he was outstanding in his field! 😂 And speaking '
             'of fields, have you ever tried standing u

In [10]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'Qwen2.5-7B-Instruct': {'completion_tokens': 185,
                                                              'cost': 0,
                                                              'prompt_tokens': 310,
                                                              'total_tokens': 495},
                                      'total_cost': 0},
 'usage_including_cached_inference': {'Qwen2.5-7B-Instruct': {'completion_tokens': 185,
                                                              'cost': 0,
                                                              'prompt_tokens': 310,
                                                              'total_tokens': 495},
                                      'total_cost': 0}}


In [11]:
# 默认情况下，使用最后一条消息作为summary
pprint.pprint(chat_result.summary)

("You go, Joe! Flexibility is key, whether you're spinning a joke or a plate. "
 'Here’s one for you: Why did the scarecrow win an award? Because he was '
 'outstanding in his field! 😂 And speaking of fields, have you ever tried '
 "standing up on a field full of cows? It's like trying to balance on a team "
 'of dairy professionals! 🐄😊')


## Get a better summary of the conversation

In [12]:
# 使用模型生成摘要
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Glad to keep the laughs coming. Why did the tomato turn red? Because it saw the salad dressing! 😂 Now, are you a leftie or a rightie? Because I heard lefties have an advantage in stand-up comedy because they're used to dealing with all the awkward left-handed equipment! 😜

--------------------------------------------------------------------------------
joe (to cathy):

Well, I'm right-handed, but I guess I must be pretty flexible because I can make both left-handed and right-handed jokes! 🤚✍️ 👉✍️

--------------------------------------------------------------------------------
cathy (to joe):

You go, Joe! Flexibility is key, whether you're spinning a joke or a plate. Here’s one for you: Why did the scarecrow win an award? Because he was outstanding in his field! 😂 And speaking of fields, have you ever tried standing 

In [13]:
pprint.pprint(chat_result.summary)

{'audio': None,
 'content': 'Joe is right-handed but flexible enough to make jokes for both '
            'lefties and righties. He enjoys humor and playful banter. A '
            "couple of jokes about being outstanding in one's field and "
            'balancing on a field of cows were shared.',
 'function_call': None,
 'refusal': None,
 'role': 'assistant',
 'tool_calls': []}


## Chat Termination

Chat can be terminated using a termination conditions.

In [12]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [13]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Let's get this comedy show on the road. Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! I bet he was also great at hay-ing low.

--------------------------------------------------------------------------------
cathy (to joe):

Haha, I see what you did there, Joe! That scarecrow knows how to make a great first “straw-tion!”

--------------------------------------------------------------------------------
joe (to cathy):

Haha, nice one, Cathy! That scarecrow sure knows how to make a grand entrance. Speaking of which, have you heard about the actor who fell through the floorboards? He was just going through a stage.

----------------------------------------------

In [14]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

We were talking about the actor who fell through the floorboards and made a dramatic exit!

--------------------------------------------------------------------------------
cathy (to joe):

Thanks for jogging my memory! That was a fun one. If you want to hear more jokes or chat again, feel free to reach out. Have a great day!

--------------------------------------------------------------------------------
joe (to cathy):

You too, Cathy! Have a fantastic day ahead!

--------------------------------------------------------------------------------
cathy (to joe):

Thanks, Joe! Take care and keep smiling! Byeee!

--------------------------------------------------------------------------------
joe (to cathy):

Bye!

--------------------------------------------------------------------------------
cathy (to joe):

Goodbye!

-------------------